In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import cv2
from cifar10_models import mobilenetv2

In [2]:

Net = mobilenetv2.mobilenet_v2(pretrained=True)

In [13]:
for param in Net.parameters():
    param.requires_grad = False

In [16]:
class RegressionModel(nn.Module):
    def __init__(self, pretrainedModel):
        super().__init__()
        self.features = pretrainedModel.features
        
        self.flatten = nn.Flatten()
        
        self.Dense1 = nn.Linear(20480, 32)

        self.outLayer1 = nn.Linear(32, 4)
        self.outLayer2 = nn.Linear(32, 4)
        self.outLayer3 = nn.Linear(32, 4)
        self.outLayer4 = nn.Linear(32, 4)
    
    def forward(self, x):
        x = self.features(x)
        x = self.Dense1(self.flatten(x))

        out1 = self.outLayer1(x)
        out2 = self.outLayer2(x)
        out3 = self.outLayer3(x)
        out4 = self.outLayer4(x)

        return out1, out2, out3, out4


In [17]:
newModel = RegressionModel(Net)

In [34]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.Layer1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)

        self.Layer2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
    
        self.Flatten = nn.Flatten()

        self.Dense1 = nn.Linear(2048, 32)

        self.outLayer1 = nn.Linear(32, 4)
        self.outLayer2 = nn.Linear(32, 4)
        self.outLayer3 = nn.Linear(32, 4)
        self.outLayer4 = nn.Linear(32, 4)

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.Layer1(x)), 2)
        out = F.max_pool2d(torch.relu(self.Layer2(out)), 2)
        out = self.Flatten(out)
        out = torch.relu(self.Dense1(out))
        
        out1 = self.outLayer1(out)
        out2 = self.outLayer2(out)
        out3 = self.outLayer3(out)
        out4 = self.outLayer4(out)
        return out1, out2, out3, out4

In [40]:
model = Model()

In [10]:
img = cv2.imread('data/attacks/00000_input_img.png')
imgT = torch.tensor(img)
imgT.shape

torch.Size([32, 32, 3])

In [11]:
imgT = imgT.view(3, 32 , 32)
imgT = torch.unsqueeze(imgT, 0).float()

In [32]:
imgT.dtype

torch.float32

In [18]:
newModel.forward(imgT)

False
True
True


(tensor([[-0.0662, -0.1314,  0.1177,  0.1573]], grad_fn=<AddmmBackward>),
 tensor([[ 0.1317, -0.0049,  0.0881,  0.1280]], grad_fn=<AddmmBackward>),
 tensor([[-0.1416, -0.0165,  0.1029,  0.0024]], grad_fn=<AddmmBackward>),
 tensor([[0.1285, 0.0106, 0.1514, 0.1382]], grad_fn=<AddmmBackward>))

In [41]:
model.forward(imgT)

(tensor([[-5.8843, -5.0632, -2.5045,  0.7898]], grad_fn=<AddmmBackward>),
 tensor([[ 1.2375, -5.6838,  1.5965,  5.8334]], grad_fn=<AddmmBackward>),
 tensor([[ 6.7813, -6.6644,  3.2673,  7.8263]], grad_fn=<AddmmBackward>),
 tensor([[4.1131, 2.7572, 3.8784, 2.2712]], grad_fn=<AddmmBackward>))